# Protocol Data Blob 

Goal: load protocol data blob in from matlab & unpack

Protocol Data structure for DMS:

| variable             | values          | key                                                 |
|----------------------|-----------------|-----------------------------------------------------|
| `hit_history`        | [0, 1, NaN]     | [error, hit, violation]                             |
| `temp_error_history` | [0, 1, NaN]     | [no terror, terror, violation]                      |
| `side_history`       | [l, r]          | [left, right]                                       |
| `result_history`     | [1, 2, 3, 4, 5] | [hit, error, violation, terror, crash]              |
| `sa`                 | float           | value of sa in Hz                                   |
| `sb`                 | float           | value of sb in Hz                                   |
| `dms_type`           | [0, 1]          | [nonmatch, match]                                   |
| `delay`              | float           | duration of delay between sa/sb in s                |
| `fixation`           | float           | duration of pre --> go (fixation required or not)   |
| `timeouts`           | float           | duration of timeout in s                            |
| `helper`             | [0, 1]          | if trial was in helper block (not in use currently) |
| `stage`              | float           | stage number                                        |


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import datajoint as dj
import pandas as pd
import os
import seaborn as sns
import pandas as pd

from io_utils import *

# settings
%load_ext autoreload
%autoreload 
dj.blob.use_32bit_dims = True # necessary for pd.blob read
 

### Single Session

In [3]:
# grab protocol data from a session
bdata = dj.create_virtual_module('new_acquisition', 'bdatatest')
session_key = {'sessid': 880487}
session_data = (bdata.Sessions & session_key).fetch('protocol_data', as_dict=True)

[2022-08-10 12:47:05,922][INFO]: Connecting jbreda@datajoint01.pni.princeton.edu:3306
[2022-08-10 12:47:06,291][INFO]: Connected jbreda@datajoint01.pni.princeton.edu:3306


In [4]:
animal_id = 'R500'
sessid =  session_key['sessid']
date = "2022-07-19"
protocol_dict = mymblob_to_dict(session_data[0]['protocol_data'])
protocol_df = make_protocol_df(protocol_dict, animal_id, date, sessid)

protocol_df.head()

updating sb values to be correct


,trial,animal_id,date,sessid,hits,temperror,sides,result,sa,sb,dms_type,delay,fixation,timeouts,helper,stage
0,1,R500,2022-07-19,880487,1,0,r,1,12.0,3.0,False,0.1,1.100000,NaN,0,4
1,2,R500,2022-07-19,880487,<NA>,<NA>,r,3,12.0,3.0,False,0.1,0.455246,0.500,0,4
2,3,R500,2022-07-19,880487,<NA>,<NA>,r,3,12.0,3.0,False,0.1,0.455246,0.525,0,4
3,4,R500,2022-07-19,880487,0,0,l,2,12.0,12.0,True,0.1,0.455246,0.500,0,4
4,5,R500,2022-07-19,880487,<NA>,<NA>,r,3,3.0,12.0,False,0.1,0.460230,0.550,0,4


### Multi Session

In [47]:
subject_session_key = {'ratname': 'R500'}
pds  = (bdata.Sessions & subject_session_key).fetch('protocol_data', as_dict=True)
sessids, dates = (bdata.Sessions & subject_session_key).fetch('sessid','sessiondate')

In [41]:
dicts = []
for sess in pds:
    dict = mymblob_to_dict(sess['protocol_data'])
    dicts.append(dict)

In [49]:
len(dicts), len(sessids), len(dates)

(23, 23, 23)

#### scraps

In [157]:
for key in protocol_dict.keys():
    print(f"{key} type:{type(protocol_dict[key][0])} ")

hits type:<class 'numpy.float64'> 
temperror type:<class 'numpy.float64'> 
sides type:<class 'str'> 
result type:<class 'numpy.float64'> 
sa type:<class 'numpy.float64'> 
sb type:<class 'numpy.float64'> 
dms_type type:<class 'numpy.bool_'> 
delay type:<class 'numpy.float64'> 
fixation type:<class 'numpy.float64'> 
timeouts type:<class 'numpy.float64'> 
helper type:<class 'numpy.float64'> 
stage type:<class 'numpy.float64'> 


In [156]:
for key,value in protocol_dict.items():

    print(f"{key} length:{len(value)}")

hits length:45
temperror length:45
sides length:45
result length:45
sa length:45
sb length:45
dms_type length:45
delay length:45
fixation length:45
timeouts length:45
helper length:45
stage length:45
